Cal Poly Professor ratings scraped from https://polyratings.dev/search/name. Data collected + example:

| LastName | FirstName | Department | Rating | NumRatings |
| :------- | :-------- | :--------- | :----: | :--------: |
| Doe      | John      | CSC        | 3.5    | 24         |

In [14]:
!pip install itables
import pandas as pd
import requests
import itables
import json
from bs4 import BeautifulSoup
import re

In [3]:
# df_ratings = pd.read_json('professor_ratings.json')
# itables.show(df_ratings)

with open('professor_ratings.json') as f:
    data = json.load(f)

df_ratings = pd.json_normalize(data['data'])
columns = ["lastName", "firstName", "department", "numEvals", "overallRating", "materialClear", "studentDifficulties", "courses"]
df_ratings = df_ratings[columns]

for index, row in df_ratings[(df_ratings['firstName'].str.split().str.len()) > 1].iterrows():
    name = row['firstName']
    firstname, middlename = name.split()
    df_ratings.loc[index, 'firstName'] = firstname

itables.show(df_ratings[columns])

In [4]:
import pandas as pd

df_payroll = pd.read_csv('salaries.csv')

#Extracting first and last name from payroll dataframe to match polyrating name format
for index, row in df_payroll.iterrows():
    name = row['name']
    lastname, firstname_middle = name.split(", ")
    firstname_parts = firstname_middle.split()
    firstname = firstname_parts[0]
    df_payroll.loc[index, 'firstName'] = firstname
    df_payroll.loc[index, 'lastName'] = lastname

df_payroll['avgPay'] = df_payroll['avgPay'].round(2)

In [5]:
df_payroll

,name,avgPay,firstName,lastName
0,"Aagaard, Sofie J",91650.96,Sofie,Aagaard
1,"Abba, Michele M",57789.41,Michele,Abba
2,"Abelli-Amen, Ella Carol",4778.25,Ella,Abelli-Amen
3,"Abeloos, Diego V.",67907.67,Diego,Abeloos
4,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby
...,...,...,...,...
4728,"Zook, Garet R",63830.44,Garet,Zook
4729,"Zumalt, Julia Mae",3311.86,Julia,Zumalt
4730,"Zumbro, Emma Catherine",6752.28,Emma,Zumbro
4731,"Zuzow, Lynnetta Robin",68914.67,Lynnetta,Zuzow


In [6]:
print("The number of payroll is " + str(len(df_payroll)))
print("The number of rating is " + str(len(df_ratings)))
df_payRating = df_payroll.merge(df_ratings, on=["firstName", "lastName"])

The number of payroll is 4733
The number of rating is 1862


In [8]:
df_payRating

,name,avgPay,firstName,lastName,department,numEvals,overallRating,materialClear,studentDifficulties,courses
0,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby,AERO,20,3.39,3.34,3.42,"[AERO 101, AERO 121, MATH 142, AERO 300, AERO ..."
1,"Abney, Keith A",99122.78,Keith,Abney,PHIL,138,2.31,2.61,2.05,"[PHIL 124, PHIL 125, PHIL 126, PHIL 145, PHIL ..."
2,"Adams, Nikki L",116394.67,Nikki,Adams,BIO,27,2.82,2.68,2.75,"[BIO 160, BIO 336, ZOO 336, BIO 405, BIO 440, ..."
3,"Adan, Elizabeth",130039.99,Elizabeth,Adan,ART,37,3.56,3.66,3.48,"[ART 112, ART 118, ART 311, ART 312, ART 315, ..."
4,"Agbo, Samuel O",85824.17,Samuel,Agbo,EE,80,1.23,1.28,1.01,"[EE 112, EE 151, EE 211, EE 212, EE 228, EE 24..."
...,...,...,...,...,...,...,...,...,...,...
902,"Zhang, Shanju",117869.34,Shanju,Zhang,CHEM,25,2.88,2.81,2.77,"[CHEM 124, CHEM 125, CHEM 127, CHEM 128, CHEM ..."
903,"Zigler, David F",95367.33,David,Zigler,CHEM,13,1.16,1.23,1.38,"[CHEM 121, CHEM 126, CHEM 127, CHEM 124, CHEM ..."
904,"Zoerb, Matthew C",98424.00,Matthew,Zoerb,CHEM,1,3.00,3.00,3.00,[CHEM]
905,"Zoller, David J",74328.67,David,Zoller,PHIL,13,3.23,3.08,3.08,"[PHIL 230, PHIL 231, PHIL 331, PHIL 335]"


In [23]:
headers = {
    'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    'Accept-Language': 'en-US,en;q=0.5'
}

url = "https://catalog.calpoly.edu/facultyandstaff/#facultystaffemeritustext"

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

doc = soup.find('table', class_='tbl_facdir')
rows = doc.find_all('tr')

data = []
for row in rows[1:]:  # Skip the header row
    columns = row.find_all('td')
    name_position = columns[0].get_text(separator=" ").strip()
    position = columns[1].get_text(separator=" ").strip()
    education = columns[2].get_text(separator=" ").strip()

    # Splitting name and position
    if '(' in name_position:
        name = name_position.split('(')[0].strip()
        dept = name_position.split('(')[1].replace(')', '').strip()
    else:
        name = name_position
        dept = ''

    data.append([name, position, dept, education])

df_professor = pd.DataFrame(data, columns=['Name', 'Position', 'Department', 'Education'])

In [24]:
df_professor['Department'] = df_professor['Department'].apply(lambda x: ' '.join(x.split()[1:]))
df_professor

,Name,Position,Department,Education
0,"Abercromby, Kira J.",Professor,Aerospace Engineering,"B.S., University of California, Los Angeles, 1..."
1,"Abo Ismail, Mohammed",Assistant Professor,Animal Science,"B.S., Alexandria University Damanhour, 2000; M..."
2,"Adams, Nikki L.",Professor,Biological Sciences,"B.A., University of California, Santa Barbara,..."
3,"Adams, Aubrie S.",Associate Professor,Communication Studies,"B.A., Sonoma State University, 2005; M.A., Sac..."
4,"Adams, Elizabeth",Assistant Professor,Construction Management,"B.S., Boise State University, 2001; M.S., Univ..."
...,...,...,...,...
773,"Zhang, Pei",Assistant Professor,Experience Industry Management,"B.S., Wuhan University of Science and Technolo..."
774,"Zhang, Alan",Assistant Professor,Mechanical Engineering,"B.S. University of California, Berkeley 2016, ..."
775,"Zigler, David F.",Associate Professor,Chemistry and Biochemistry,"B.S., Eastern Illinois University; Ph.D., Virg..."
776,"Zoerb, Matthew C.",Associate Professor,Chemistry and Biochemistry,"B.S., UT Chattanooga, 2006; Ph.D., Univeristy ..."


In [25]:
# Extracting first and last name for each professor
for index, row in df_professor.iterrows():
    name = row['Name']
    lenName = len(name.split(", "))
    if lenName == 2:
      lastname, firstname_middle = name.split(", ")
    elif lenName == 3:
      lastname, firstname_middle, _ = name.split(", ")
      print(name)
    elif lenName == 4:
      lastname, firstname_middle, _, _ = name.split(", ")
      print(name)
    firstname_parts = firstname_middle.split()
    firstname = firstname_parts[0]
    df_professor.loc[index, 'firstName'] = firstname
    df_professor.loc[index, 'lastName'] = lastname

df_professor

Castilow, Dan C., II
Fowler, Thomas , IV, FAIA


,Name,Position,Department,Education,firstName,lastName
0,"Abercromby, Kira J.",Professor,Aerospace Engineering,"B.S., University of California, Los Angeles, 1...",Kira,Abercromby
1,"Abo Ismail, Mohammed",Assistant Professor,Animal Science,"B.S., Alexandria University Damanhour, 2000; M...",Mohammed,Abo Ismail
2,"Adams, Nikki L.",Professor,Biological Sciences,"B.A., University of California, Santa Barbara,...",Nikki,Adams
3,"Adams, Aubrie S.",Associate Professor,Communication Studies,"B.A., Sonoma State University, 2005; M.A., Sac...",Aubrie,Adams
4,"Adams, Elizabeth",Assistant Professor,Construction Management,"B.S., Boise State University, 2001; M.S., Univ...",Elizabeth,Adams
...,...,...,...,...,...,...
773,"Zhang, Pei",Assistant Professor,Experience Industry Management,"B.S., Wuhan University of Science and Technolo...",Pei,Zhang
774,"Zhang, Alan",Assistant Professor,Mechanical Engineering,"B.S. University of California, Berkeley 2016, ...",Alan,Zhang
775,"Zigler, David F.",Associate Professor,Chemistry and Biochemistry,"B.S., Eastern Illinois University; Ph.D., Virg...",David,Zigler
776,"Zoerb, Matthew C.",Associate Professor,Chemistry and Biochemistry,"B.S., UT Chattanooga, 2006; Ph.D., Univeristy ...",Matthew,Zoerb


In [26]:
df_payRating = df_payRating.merge(df_professor, on=["firstName", "lastName"], how='left')
df_payRating = df_payRating.drop(columns=['Name'], axis=0)

In [27]:
df_payRating['Position'].fillna('Lecturer/TA', inplace=True)

In [28]:
df_payRating

,name,avgPay,firstName,lastName,department,numEvals,overallRating,materialClear,studentDifficulties,courses,Position,Department,Education
0,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby,AERO,20,3.39,3.34,3.42,"[AERO 101, AERO 121, MATH 142, AERO 300, AERO ...",Professor,Aerospace Engineering,"B.S., University of California, Los Angeles, 1..."
1,"Abney, Keith A",99122.78,Keith,Abney,PHIL,138,2.31,2.61,2.05,"[PHIL 124, PHIL 125, PHIL 126, PHIL 145, PHIL ...",Lecturer/TA,NaN,NaN
2,"Adams, Nikki L",116394.67,Nikki,Adams,BIO,27,2.82,2.68,2.75,"[BIO 160, BIO 336, ZOO 336, BIO 405, BIO 440, ...",Professor,Biological Sciences,"B.A., University of California, Santa Barbara,..."
3,"Adan, Elizabeth",130039.99,Elizabeth,Adan,ART,37,3.56,3.66,3.48,"[ART 112, ART 118, ART 311, ART 312, ART 315, ...",Professor and WGQS Department Chair,"Women's, Gender and Queer Studies, Interdiscip...","B.A., University of California, Davis, 1993; M..."
4,"Agbo, Samuel O",85824.17,Samuel,Agbo,EE,80,1.23,1.28,1.01,"[EE 112, EE 151, EE 211, EE 212, EE 228, EE 24...",Professor,Electrical Engineering,"B.Sc., University of Nigeria, 1975; M.S.E., Un..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,"Zhang, Shanju",117869.34,Shanju,Zhang,CHEM,25,2.88,2.81,2.77,"[CHEM 124, CHEM 125, CHEM 127, CHEM 128, CHEM ...",Professor,Chemistry and Biochemistry,"B.S., Jilin University, People's Republic of C..."
903,"Zigler, David F",95367.33,David,Zigler,CHEM,13,1.16,1.23,1.38,"[CHEM 121, CHEM 126, CHEM 127, CHEM 124, CHEM ...",Associate Professor,Chemistry and Biochemistry,"B.S., Eastern Illinois University; Ph.D., Virg..."
904,"Zoerb, Matthew C",98424.00,Matthew,Zoerb,CHEM,1,3.00,3.00,3.00,[CHEM],Associate Professor,Chemistry and Biochemistry,"B.S., UT Chattanooga, 2006; Ph.D., Univeristy ..."
905,"Zoller, David J",74328.67,David,Zoller,PHIL,13,3.23,3.08,3.08,"[PHIL 230, PHIL 231, PHIL 331, PHIL 335]",Associate Professor,Philosophy,"B.A. Xavier University, 2003; M.A., Fordham Un..."


In [34]:
url = "https://web.archive.org/web/20240116141031/https://www.polyratings.com/list.html"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find all the links to professors' evaluations
doc = soup.find_all('a', class_='no-link-highlight text-muted filterable')

# Initialize a list to hold the data
data = []

pattern = r"([a-zA-Z\s,]+)(\d+\.\d+)"

# Iterate over the links and extract the required information
for row in doc:
    name_rating = row.find('strong', class_='hidden-xs').get_text(strip=True)

    match = re.match(pattern, name_rating)
    if match:
        name = match.group(1).strip()
        rating = float(match.group(2))
    else:
        name = name_rating.strip()
        rating = None

    lenName = len(name.split(", "))
    if lenName == 2:
      lastname, firstname = name.split(", ")
    elif lenName == 3:
      lastname, firstname, _ = name.split(", ")
      print(name)
    elif lenName == 4:
      lastname, firstname, _, _ = name.split(", ")
      print(name)

    department = row.find('h4').get_text(strip=True).split()[0]
    evals_text = row.find('h4').find('span', class_='pull-right').get_text(strip=True)

    # Handle non-numeric evaluations
    try:
        evals = int(evals_text.split()[0])
    except ValueError:
        evals = None

    # Append the extracted data to the list
    data.append([name, firstname, lastname, rating, department, evals])

# Convert the list to a DataFrame
df_archiveRating = pd.DataFrame(data, columns=['Name', 'firstName', 'lastName', 'Rating', 'Department', 'Number of Evaluations'])
df_archiveRating

Bailey, Jr., Philip2.71


,Name,firstName,lastName,Rating,Department,Number of Evaluations
0,"Abel, Christina",Christina,Abel,3.22,CHEM37,37
1,"Aberbromby, Kira",Kira,Aberbromby,3.46,AERO13,13
2,"abercromby, kira",kira,abercromby,3.22,AERO9,9
3,"Abitia, Fred",Fred,Abitia,3.61,IT18,18
4,"Abler, Fred",Fred,Abler,1.75,ARCH8,8
...,...,...,...,...,...,...
2473,"Zohns, Michael",Michael,Zohns,3.88,EHS10,10
2474,"Zoller, David",David,Zoller,3.09,PHIL11,11
2475,"Zulfacar, Maliha",Maliha,Zulfacar,1.99,SOC93,93
2476,"Zumwalt, Marta",Marta,Zumwalt,2.77,CHEM13,13


In [35]:
df_archiveRating = df_archiveRating.merge(df_payroll, on=["firstName", "lastName"], how="left")
df_archiveRating

,Name,firstName,lastName,Rating,Department,Number of Evaluations,name,avgPay
0,"Abel, Christina",Christina,Abel,3.22,CHEM37,37,NaN,NaN
1,"Aberbromby, Kira",Kira,Aberbromby,3.46,AERO13,13,NaN,NaN
2,"abercromby, kira",kira,abercromby,3.22,AERO9,9,NaN,NaN
3,"Abitia, Fred",Fred,Abitia,3.61,IT18,18,NaN,NaN
4,"Abler, Fred",Fred,Abler,1.75,ARCH8,8,NaN,NaN
...,...,...,...,...,...,...,...,...
2481,"Zohns, Michael",Michael,Zohns,3.88,EHS10,10,NaN,NaN
2482,"Zoller, David",David,Zoller,3.09,PHIL11,11,"Zoller, David J",74328.67
2483,"Zulfacar, Maliha",Maliha,Zulfacar,1.99,SOC93,93,NaN,NaN
2484,"Zumwalt, Marta",Marta,Zumwalt,2.77,CHEM13,13,NaN,NaN
